In [1]:
#We will now look at another popular type of database whichc is NoSQL.

#Mongita is a file based NoSQL database.

#SQL vs NoSQL

#NoSQL Database are not an competition or alternative to SQL database. There are somethings that SQL database does better while there some things that NoSQL databases does better.
# In the real world NoSQL databases compliment SQL databases and provide a complete solution for data management and persistance requirements.
# Some NoSQL databases also use SQL as query language.

#What is NoSQL database?
# Traditional relational databases as we've seen, uses tabular data structures with common keys creating a relation between them. 
# On the other hand NoSQL databases can be of various types.
# MongoDB -  Document type to store data
# Cassandra - Column oriented, i.e., it stores columns to store data instead of conventional rows.
# Redis - Key Value pair cached in memory, extemely fast to query and update.
# Neo4j -  Graph Databaase. Nodes with data and edges with relationships

#Basic concepts of MongoDb.

# A database is a group of collections
# A collection is a group of documents.
# A document stores data (conceptually) in JSON, actually stored in BSON (Binary).

#Mongo DB document is schemaless. That means it does not care if a document in a collection have the same structure or not. Unlike SQL databases which have columns that every row needs to fill.



In [2]:
#Just as SQLite is a file based loal database for a SQL relational database.
#Mongita is a file based NoSQL Database based on MongoDB

#SQLLite is the database specification that works with sqlite command line or sqlite3 package in python. 
# Mongita on the other hand is python package that implements a subset of the pymongo api. Pymongo is the complete MongoDB api and offical packagge for mongodb

# Just like SQLite, mongoita is used as an embedded database and is also used for unit testing code.

#Install mongita
%pip install mongita

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/949.4 kB ? eta -:--:--
   --------------------------------------- 949.4/949.4 kB 45.1 MB/s eta 0:00:00
  Created wheel for mongita: filename=mongita-1.2.0-py3-none-any.whl size=61386 sha256=3486f224ca378f33c6e95e3fc5fbdf7010d6b439fb4021c4452ca056cb057cb4
  Stored in directory: c:\users\adminuser\appdata\local\pip\cache\wheels\d9\68\e4\98c564b08c1290c65bfd8f6b13d8599ef33b4f53e6671e5b1d
Successfully built mongita

   ---------- ----------------------------- 1/4 [dnspython]
   ---------- ----------------------------- 1/4 [dnspython]
   ---------- ----------------------------- 1/4 [dnspython]
   ---------- ------------

In [5]:
from mongita import MongitaClientDisk

#Initialize a mongita client, which in every way should also work with pymongo api for mongodb
client = MongitaClientDisk()

#The database is a dynamic attribute of the client
db = client.portfolio #get or create the portfolio database

#The collections are dynamic attributes of the the database.
investments = db.investments #get or create a collection.

#Adding an entry to the collection, using the inster_one function.
investments.insert_one({ #Remember that documents in mongodb are nothing but JSON entries, and JSON, structurally and syntactically are similar to Python objects
    "coin_id": "bitcoin",
    "currency": "USD",
    "amount": 1.0
}) #there was no need to define any schema beforehand, you can input any data, as long as it is a valid python dictionary.

#insert many entries into the collection.
investments.insert_many([
    {"coin_id": "ethereum", "currency": "USD", "amount": 10.0},
    {"coin_id": "solana", "currency": "USD", "amount": 25.0}
])

In [21]:
#get all documents with a coin_id of bitcoin

bitcoin_investments = investments.find({"coin_id": "bitcoin"}) #Key point to note here is the filter that the find() took was again a document, whcih is a python object, we did not have to write SQL queries.
[print(investment) for investment in bitcoin_investments]
print("--------------------------------------------")
#get first document in the collection matching the filter
bitcoin_investments = investments.find_one({"coin_id": "bitcoin"})
print(bitcoin_investments)
print("--------------------------------------------")
#Get all documents in the collection
bitcoin_investments = investments.find({}) # empty filter results in an all documents
[print(investment["coin_id"]) for investment in bitcoin_investments]

{'_id': ObjectId('684482c32985e8e5d15b3ba2'), 'coin_id': 'bitcoin', 'currency': 'USD', 'amount': 1.6000000000000005}
{'_id': ObjectId('684482d02985e8e5d15b3ba5'), 'coin_id': 'bitcoin', 'currency': 'USD', 'amount': 1.6000000000000005}
{'coin_id': 'bitcoin', 'currency': 'USD', 'amount': 1.6000000000000005, '_id': ObjectId('684483022985e8e5d15b3ba6')}
--------------------------------------------
{'_id': ObjectId('684482c32985e8e5d15b3ba2'), 'coin_id': 'bitcoin', 'currency': 'USD', 'amount': 1.6000000000000005}
--------------------------------------------
bitcoin
bitcoin
bitcoin
solana


[None, None, None, None]

In [ ]:
#Updating records

# Increment amount of each bitcoin entry by 0.1
updates = investments.update_many( #Just like insert, update has two methods update_one and update_many
    {"coin_id": "bitcoin"}, #The filter document to find the entries
    {"$inc": {"amount": 0.1}}    # The $inc is an increment operator that applies the {"amount": 0.1} document on the documents fildtered by the filter document.
)
[print(investment) for investment in investments.find({})]

print("--------------------------------------------")

#Delete document
deletes = investments.delete_one({"coin_id": "ethereum"})
[print(investment.) for investment in investments.find({})]


{'_id': ObjectId('684482c32985e8e5d15b3ba2'), 'coin_id': 'bitcoin', 'currency': 'USD', 'amount': 1.6000000000000005}
{'_id': ObjectId('684482d02985e8e5d15b3ba5'), 'coin_id': 'bitcoin', 'currency': 'USD', 'amount': 1.6000000000000005}
{'coin_id': 'bitcoin', 'currency': 'USD', 'amount': 1.6000000000000005, '_id': ObjectId('684483022985e8e5d15b3ba6')}
{'coin_id': 'solana', 'currency': 'USD', 'amount': 25.0, '_id': ObjectId('684483022985e8e5d15b3ba8')}
--------------------------------------------


NameError: name 'coin_id' is not defined

In [ ]:
#Mongita implementation of the crypto investment manager app.

import requests
import click
from mongita import MongitaClientDisk

api_key = "CG-YgaSDYyvsiDQsZvTB8KNjJcR"

headers = {
    "x-cg-demo-api-key": api_key
}


# Establish database connection
client = MongitaClientDisk()
database = client.new_portfolio
investments = database.investments


#CLick is the pythonic way to create a command line interface for your scripts or application. It's smart and easy to use and creates arguments automatically for the decorators defined.
# Under the hood it has a CLI API that has a commandline parser that talks with the interpreter than works completely different than optparse or argparse built in libraries. It gives you easy ways to create highly nested and complex commands with options.
# CLick also provides out of the box support for reading env variables form values and lazy loading and file handling.
# You transform methods into click commands using decorators.

#Create a command group, a group can have multiple commands assocated with it. The group name will the function name
@click.group()
def cli():
    pass

def get_coin_price(coin_id, currency):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies={currency}"
    data = requests.get(url, headers=headers).json()
    return data[coin_id][currency]

@cli.command() #Command decorator that aslo assigns this command to the group.
@click.option("--coin_id", default="bitcoin") #Adding an option
@click.option("--currency", default="usd")
def get_investment_value(coin_id, currency): #the option name gets passed onto the function parameter name. Then the function uses iut wthin the code.
    coin_price = get_coin_price(coin_id, currency)
    filter_doc_buy = {
        "coin_id": coin_id,
        "currency": currency,
        "sell": "False"
    }
    filter_doc_sell = {
        "coin_id": coin_id,
        "currency": currency,
        "sell": "True"
    }
    buy_result = investments.find(filter_doc_buy)
    sell_result = investments.find(filter_doc_sell)
    
    buy_amount = sum(row["amount"] for row in buy_result) 
    sell_amount = sum(row["amount"] for row in sell_result)
    
    total = buy_amount - sell_amount
    
    print(f"You own a total of {total} {coin_id} worth {total * coin_price} {currency.upper()}")

@cli.command()
@click.option("--coin_id", default="bitcoin")
@click.option("--currency", default="usd")
def show_coin_price(coin_id, currency):
    coin_price = get_coin_price(coin_id, currency)
    print(f"The price of {coin_id} is {coin_price:.2f} {currency.upper()}")
    
@cli.command()
@click.option("--coin_id")
@click.option("--currency")
@click.option("--amount", type=float)
@click.option("--sell", is_flag=True)
def add_investments(coin_id, currency, amount, sell):
    try:
        # sql = "INSERT INTO investments (coin_id, currency, amount, sell, date) VALUES (?, ?, ?, ?, ?);"
        investments.insert_one({
            "coin_id": coin_id,
            "currency": currency,
            "amount": amount,
            "sell": f"{True if sell else False}"
        })
        print(f"Added {'sell' if sell else 'buy'} of {amount} {coin_id}")
    except Exception as e:
        print(f"An error occurred: {e}")
        
@cli.command()
def show_db():
    results = investments.find({})
    [print(result) for result  in results]

# Function to run commands in Jupyter
def run_command(command, *args):
    try:
        cli.main(args=[command] + list(args), standalone_mode=False)
    except SystemExit:
        pass

# Example usage in Jupyter
run_command('show-coin-price', '--coin_id', 'ethereum', '--currency', 'usd')
run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '1.0')
#selling some btc
run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '0.5', '--sell')

#Getting investment value
run_command('get-investment-value', '--coin_id', 'bitcoin', '--currency', 'inr')

run_command('show-db')

The price of ethereum is 2520.52 USD
Added buy of 1.0 bitcoin
Added sell of 0.5 bitcoin
You own a total of 2.0 bitcoin worth 18150686.0 INR
{'_id': ObjectId('684499c52985e8e5d15b3bae'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 1.0, 'sell': 'False'}
{'_id': ObjectId('684499c52985e8e5d15b3baf'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 0.5, 'sell': 'True'}
{'_id': ObjectId('684499d22985e8e5d15b3bb0'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 1.0, 'sell': 'False'}
{'_id': ObjectId('684499d22985e8e5d15b3bb1'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 0.5, 'sell': 'True'}
{'_id': ObjectId('684499dd2985e8e5d15b3bb2'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 1.0, 'sell': 'False'}
{'_id': ObjectId('684499dd2985e8e5d15b3bb3'), 'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 0.5, 'sell': 'True'}
{'coin_id': 'bitcoin', 'currency': 'inr', 'amount': 1.0, 'sell': 'False', '_id': ObjectId('68449cef2985e8e5d15b3bb4')}
{'coin_id': 'bitcoin', 'curren

In [27]:
str(True)

'True'